# Mapping using bwa-mem

Was considering using bwa-mem2 (https://github.com/bwa-mem2/bwa-mem2). It's much faster but needs much more memory. We don't have a speed problem so not needed

use preferred conda env  
**Packages needed**: bwa-mem, samtools

In [1]:
import sys
import ipyparallel as ipp
import os
from os import environ
import gzip
import warnings
import pandas as pd
import numpy as np
import scipy as sp
import glob
import re
import random

In [2]:
root = "/data/gpfs/assoc/denovo/tfaske/SPCR/"

In [3]:
cd $root

/data/gpfs/assoc/denovo/tfaske/SPCR


In [4]:
fq_dir = '/data/gpfs/assoc/denovo/tfaske/SPCR/fastq'

In [5]:
pwd

'/data/gpfs/assoc/denovo/tfaske/SPCR'

## Assembly

with dDocent c=.96, k1=8, k2=6

In [6]:
assembly = os.path.join(root,"assembly/reference.fasta")

In [7]:
!samtools faidx $assembly

## Actual Mapping 
 

In [8]:
fastq_files = []
files = !find $fq_dir -name '*fq.gz'
files = [os.path.abspath(x) for x in files]
for x in files:
    fastq_files.append(x)
fastq_files = sorted(fastq_files)

In [9]:
len(fastq_files),fastq_files[0]

(626, '/data/gpfs/assoc/denovo/tfaske/SPCR/fastq/SE10_F_1.fq.gz')

In [10]:
cd $root

/data/gpfs/assoc/denovo/tfaske/SPCR


In [29]:
# -k INT minimum seed length [19]
# -w INT band width for banded alignment [100]
# -r FLOAT look for internal seeds inside a seed longer than {-k} * FLOAT [1.5]
# -T INT minimum score to output [30]
# -R STR read group header line such as '@RG\tID:foo\tSM:bar' [null]

#@lview.remote()
def run_bwamem(args):
    import os, multiprocessing, socket
    cpus = 1
    assembly, fq, outdir = args
    ID = fq.split('/')[8] ### need to change this to match your ID 
    ID = ID.split('.fq.')[0] ### This too 
    sam = os.path.join(outdir, "{}.sam".format(os.path.basename(ID)))
    bam = sam.replace('.sam','.bam')
    bam_sorted = "%s_sorted.bam" % bam.replace(".bam", "")
    bwa_cmd = r"bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\tID:%s\tLB:%s\tSM:%s\tPL:ILLUMINA' %s %s > %s" % (ID,ID,ID,assembly,fq,sam)
    s2b_cmd =  "samtools view -b %s -o %s\n\nsamtools sort -@ %s %s -o %s\n\nsamtools index %s\n\n" % (sam,bam,cpus,bam,bam_sorted,bam_sorted)                                                              
    return  bwa_cmd,s2b_cmd 

In [30]:
!mkdir SNPcall

In [31]:
!mkdir SNPcall/bwa

In [32]:
!mkdir SNPcall/bwa/shdir

In [33]:
bwa_dir = os.path.join(root,"SNPcall/bwa/")
assert(bwa_dir)

In [34]:
### creates a list of commands for bwa-mem for each fastq file
res_bwa = []
res_s2b = []
for f in fastq_files:
    r1,r2 = run_bwamem((assembly, f, bwa_dir))
    res_bwa.append(r1)
    res_s2b.append(r2)

In [35]:
len(res_bwa),res_bwa[0]

(626,
 "bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\\tID:SE10_F_1\\tLB:SE10_F_1\\tSM:SE10_F_1\\tPL:ILLUMINA' /data/gpfs/assoc/denovo/tfaske/SPCR/assembly/reference.fasta /data/gpfs/assoc/denovo/tfaske/SPCR/fastq/SE10_F_1.fq.gz > /data/gpfs/assoc/denovo/tfaske/SPCR/SNPcall/bwa/SE10_F_1.sam")

In [36]:
len(res_s2b),res_s2b[0]

(626,
 'samtools view -b /data/gpfs/assoc/denovo/tfaske/SPCR/SNPcall/bwa/SE10_F_1.sam -o /data/gpfs/assoc/denovo/tfaske/SPCR/SNPcall/bwa/SE10_F_1.bam\n\nsamtools sort -@ 1 /data/gpfs/assoc/denovo/tfaske/SPCR/SNPcall/bwa/SE10_F_1.bam -o /data/gpfs/assoc/denovo/tfaske/SPCR/SNPcall/bwa/SE10_F_1_sorted.bam\n\nsamtools index /data/gpfs/assoc/denovo/tfaske/SPCR/SNPcall/bwa/SE10_F_1_sorted.bam\n\n')

In [37]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/SPCR/SNPcall/bwa


#### Below selects options for slurm submission and is a function for creating a slurm script per fastq

In [38]:
fq_ID = [fq.split('/')[8].split('.fq.')[0] for fq in fastq_files]

In [39]:
len(fq_ID), fq_ID[0]

(626, 'SE10_F_1')

In [40]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'bwa_SPCR'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 10000
email = 'tfaske@nevada.unr.edu'

In [41]:
def write_bwamem_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,fq_ID,bwa_cmds,s2b_cmds):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    for i in range(0,len(bwa_cmds)):
        with open("shdir/run_bwamem_%s.sh" % (fq_ID[i]), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_bwamem
#SBATCH --output bwa/shdir/output_bwamem_%s.txt
#SBATCH --mail-type=FAIL
#SBATCH --mail-user=%s \n\n
    
%s \n\n
%s \n""" % (account,partition,time,int(cpus),int(mem_cpu),fq_ID[i],fq_ID[i],email,bwa_cmds[i],s2b_cmds[i]))

In [42]:
write_bwamem_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,fq_ID,res_bwa,res_s2b)

#### finds all bwa slurm scripts and writes bash script to sbatch them

In [43]:
shbwa_files = []
files = !find ./shdir -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shbwa_files.append(x)
shbwa_files = sorted(shbwa_files)

In [44]:
len(shbwa_files),shbwa_files[0]

(626,
 '/data/gpfs/assoc/denovo/tfaske/SPCR/SNPcall/bwa/shdir/run_bwamem_SE10_F_1.sh')

In [45]:
def write_bash_bwamem_sh(sh_files):
    with open("SNPcall/run_bash_bwa.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))    

In [46]:
cd $root

/data/gpfs/assoc/denovo/tfaske/SPCR


In [47]:
write_bash_bwamem_sh(shbwa_files)

# Run run_bash_bwamem_sh locally
    cd /data/gpfs/assoc/denovo/PHHA/SNPcall 
    source activate py36
    bash run_bash_bwa.sh
    
## Remove sam files and unsorted bams

       whatever, run in terminal. Move sorted bams, rm bam, mv back

# Calculates coverage from bam_files (slowwwww)

also outputs a file called "bam_coverage.csv" in the bam folder

In [48]:
samtools = "samtools"

In [49]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/SPCR/SNPcall/bwa


In [50]:
bam_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    bam_files.append(x)
bam_files = sorted(bam_files)

len(bam_files), len(fastq_files)

(626, 626)

In [51]:
len(bam_files), bam_files[0]

(626, '/data/gpfs/assoc/denovo/tfaske/SPCR/SNPcall/bwa/SE10_F_10_sorted.bam')

In [52]:
bam_names = []
cov_list = []
for i in range(0,len(bam_files)):
    bam = bam_files[i]
    #print(bam)
    b = bam.split('/')[9] #set this 
    #print (b)
    !$samtools depth -a $b > $'cov.txt'
    cov = pd.read_csv('cov.txt', sep="\t",header=None)
    coverage = sum(cov.iloc[:,2])/len(cov.index)
    name = b.split('_sorted.bam')[0]
    #print(name)
    bam_names.append(name)
    cov_list.append(coverage)
    print(coverage,name,i)
cov_df = pd.DataFrame(bam=bam_names,coverage=cov_list)
cov_df.head()

16.451664426041077 SE10_F_10 0
17.88378390263977 SE10_F_11 1
18.102843412704296 SE10_F_12 2
16.88780844262109 SE10_F_1 3
13.565269247917353 SE10_F_2 4
50.03893049155146 SE10_F_3 5
25.534332133000255 SE10_F_4 6
32.63413007605694 SE10_F_5 7
16.11323962744994 SE10_F_6 8
15.70459578048576 SE10_F_7 9
23.503249821245102 SE10_F_8 10
21.889632912661632 SE10_F_9 11
20.886855360281917 SE1_F_10 12
21.65559751660867 SE1_F_11 13
14.026697738941037 SE1_F_12 14
16.97337810919863 SE1_F_1 15
18.264025259323777 SE1_F_2 16
23.554715032797336 SE1_F_3 17
25.440793527367127 SE1_F_4 18
30.73852021520713 SE1_F_5 19
20.02402831600353 SE1_F_6 20
21.340599960442674 SE1_F_7 21
16.063431532224417 SE1_F_8 22
15.69634507844457 SE1_F_9 23
18.470295370204248 SE2_F_10 24
19.178853067101485 SE2_F_11 25
12.787622752589408 SE2_F_12 26
30.704615688999557 SE2_F_1 27
20.72046187155862 SE2_F_2 28
21.418897811822514 SE2_F_3 29
16.57751549235233 SE2_F_4 30
11.06175990984973 SE2_F_5 31
14.20671954708883 SE2_F_6 32
15.22623025737

11.503933505864815 cerrillos_W_7 253
20.334394941957648 cerrillos_W_8 254
9.541080851525228 cerrillos_W_9 255
16.68030386394027 cieneguilla_W_10 256
17.702597157979852 cieneguilla_W_1 257
20.598888186354657 cieneguilla_W_2 258
10.73050810498913 cieneguilla_W_3 259
18.867488208449736 cieneguilla_W_4 260
28.202086115243084 cieneguilla_W_5 261
10.616231913616405 cieneguilla_W_6 262
18.263087388341404 cieneguilla_W_7 263
19.859738671911433 cieneguilla_W_8 264
14.356402437496556 cieneguilla_W_9 265
32.350784376582055 dell_W_10 266
26.54844700989419 dell_W_1 267
30.43921908315565 dell_W_2 268
33.827023145399636 dell_W_3 269
35.60719816778381 dell_W_4 270
13.988440367233528 dell_W_5 271
35.68769923838237 dell_W_6 272
29.080660134366482 dell_W_7 273
32.45892902058012 dell_W_8 274
34.787033433632786 dell_W_9 275
8.197999857950062 elephant_W_10 276
13.959976228945866 elephant_W_1 277
13.235490286207446 elephant_W_2 278
14.680505761634446 elephant_W_3 279
11.853359788237379 elephant_W_4 280
12.22

14.125572528403737 rda018_W_8 496
12.364415043467986 rda018_W_9 497
15.656787990852028 redhill_W_10 498
14.664175956280436 redhill_W_1 499
13.395917280123983 redhill_W_2 500
13.967037929643569 redhill_W_3 501
14.569597329115462 redhill_W_4 502
12.023910303972693 redhill_W_5 503
13.812234897238659 redhill_W_6 504
13.16628840402345 redhill_W_7 505
12.37733371810594 redhill_W_8 506
13.645183760243691 redhill_W_9 507
14.19794566813604 sandias_W_10 508
9.851313350206077 sandias_W_1 509
13.441132243789232 sandias_W_2 510
9.498634368473974 sandias_W_3 511
10.684996103119799 sandias_W_4 512
12.775304236347633 sandias_W_5 513
10.726475072419577 sandias_W_6 514
9.013761715462762 sandias_W_7 515
10.153744752513223 sandias_W_8 516
11.739618734948452 sandias_W_9 517
16.318475614451007 santacruz_W_10 518
17.989194184327253 santacruz_W_1 519
15.350834038065772 santacruz_W_2 520
13.129157761196092 santacruz_W_3 521
18.361100786079103 santacruz_W_4 522
12.53413641922348 santacruz_W_5 523
10.91615300063

TypeError: DataFrame.__init__() got an unexpected keyword argument 'bam'

In [53]:
cov_dict = {"bam":bam_names,'coverage':cov_list}
cov_df = pd.DataFrame(cov_dict)
cov_df.head()

,bam,coverage
0,SE10_F_10,16.451664
1,SE10_F_11,17.883784
2,SE10_F_12,18.102843
3,SE10_F_1,16.887808
4,SE10_F_2,13.565269


In [54]:
cov_out = os.path.join(bwa_dir,'bam_coverage.csv')
cov_df.to_csv(path_or_buf=cov_out)

In [55]:
cov_df.coverage.describe()


count    626.000000
mean      17.164251
std        6.862889
min        5.222019
25%       12.476745
50%       15.329442
75%       20.877304
max       68.797780
Name: coverage, dtype: float64

In [56]:
len(cov_df.coverage)

626

### Continue on to 4snpcalling